In [40]:
import os
import pandas as pd
import numpy as np
import json
import sys

In [41]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

sys.path.append(PARENT_DIRECTORY + '//config/')
from config import count_sequences_above_threshold, count_sequences_below_threshold, normalize_sensor_data

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [42]:
NORMALIZE_SENSOR_DATA = CONFIG_DATA['NORMALIZE_SENSOR_DATA']
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE_MOTION']
NUMBER_OF_FEATURES = CONFIG_DATA['NUMBER_OF_FEATURES']

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTION']

LABEL_COLUMN = 'Class_Motion'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTION-V0.npy'
ALL_Y_TRAIN_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTION-V0.npy'

In [43]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'STAND': 0, 'STEPS': 1, 'LSIDESTEPS': 2, 'RSIDESTEPS': 3}


In [44]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-SIDESTEPS-L-LAR-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-82BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-160BPM-AUGMENT.xl

In [45]:
def df2Xy(df, windowSize=5):
  X = []
  y_Motion= []

  # NORMALIZE THE SENSOR DATA
  if NORMALIZE_SENSOR_DATA == True:
    df["L_Pitch"] = df["L_Pitch"].apply(normalize_sensor_data)
    df["L_Roll"] = df["L_Roll"].apply(normalize_sensor_data)
    df["R_Pitch"] = df["R_Pitch"].apply(normalize_sensor_data)
    df["R_Roll"] = df["R_Roll"].apply(normalize_sensor_data)
    df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalize_sensor_data)
    df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalize_sensor_data)
    df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalize_sensor_data)
    df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalize_sensor_data)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()

        # add row values to the input
        input_data_list.append(row_values)

    # turnn list into array to do arthimetic
    input_data_array = np.array(input_data_list)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_data_array = np.abs(input_data_array)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_data_array = np.sum(input_data_array, axis=0)

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), ['Class_Motion']].values.tolist()

      y_Motion.append(label)

  return (np.array(X), np.array(y_Motion))

In [46]:
ALL_X_TRAIN_MOTION = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_MOTION = np.empty((0))  # List to store all Y training data

In [47]:
def processData(ALL_X_TRAIN_MOTION, ALL_Y_TRAIN_MOTION):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain_Motion = df2Xy(df, WINDOW_SIZE)

        yTrain_Motion = yTrain_Motion.reshape(-1)
        
        ALL_X_TRAIN_MOTION = np.concatenate([ALL_X_TRAIN_MOTION, xTrain])
        ALL_Y_TRAIN_MOTION = np.concatenate([ALL_Y_TRAIN_MOTION, yTrain_Motion])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X_TRAIN_MOTION.shape, ' ALL_Y_TRAIN_CURRVEL.shape:', ALL_Y_TRAIN_MOTION.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_motion_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y_TRAIN_MOTION)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_MOTION_PATH, ALL_X_TRAIN_MOTION)
    np.save(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH, numerical_motion_label)

In [48]:
processData(ALL_X_TRAIN_MOTION, ALL_Y_TRAIN_MOTION)

ALL_X_TRAIN_CLASSIFY_MOTION = np.load(ALL_X_TRAIN_CLASSIFY_MOTION_PATH)
ALL_Y_TRAIN_CLASSIFY_MOTION = np.load(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH)

print(ALL_X_TRAIN_CLASSIFY_MOTION.shape)
print(ALL_Y_TRAIN_CLASSIFY_MOTION.shape)

xTrain.shape: (2298, 4)
ALL_X_TRAIN.shape: (2298, 4)  ALL_Y_TRAIN_CURRVEL.shape: (2298,)
-------------------------------
xTrain.shape: (4090, 4)
ALL_X_TRAIN.shape: (6388, 4)  ALL_Y_TRAIN_CURRVEL.shape: (6388,)
-------------------------------
xTrain.shape: (3807, 4)
ALL_X_TRAIN.shape: (10195, 4)  ALL_Y_TRAIN_CURRVEL.shape: (10195,)
-------------------------------
xTrain.shape: (3147, 4)
ALL_X_TRAIN.shape: (13342, 4)  ALL_Y_TRAIN_CURRVEL.shape: (13342,)
-------------------------------
xTrain.shape: (2297, 4)
ALL_X_TRAIN.shape: (15639, 4)  ALL_Y_TRAIN_CURRVEL.shape: (15639,)
-------------------------------
xTrain.shape: (8187, 4)
ALL_X_TRAIN.shape: (23826, 4)  ALL_Y_TRAIN_CURRVEL.shape: (23826,)
-------------------------------
xTrain.shape: (7621, 4)
ALL_X_TRAIN.shape: (31447, 4)  ALL_Y_TRAIN_CURRVEL.shape: (31447,)
-------------------------------
xTrain.shape: (6301, 4)
ALL_X_TRAIN.shape: (37748, 4)  ALL_Y_TRAIN_CURRVEL.shape: (37748,)
-------------------------------
xTrain.shape: (4600,

In [49]:
ALL_X_TRAIN_CLASSIFY_MOTION = np.load(ALL_X_TRAIN_CLASSIFY_MOTION_PATH)
ALL_Y_TRAIN_CLASSIFY_MOTION = np.load(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH)

print(ALL_X_TRAIN_CLASSIFY_MOTION.shape)
print(ALL_Y_TRAIN_CLASSIFY_MOTION[:10000])

(333141, 4)
[2 2 2 ... 2 2 2]


In [50]:
print(np.max(ALL_X_TRAIN_CLASSIFY_MOTION))

154.0
